# Notebook for Integrating Reboot Motion Data with MuJoCo

__[CoLab Notebook Link](https://githubtocolab.com/RebootMotion/reboot-toolkit/blob/main/examples/RebootMotionMuJoCo.ipynb)__

Run the cells in order, making sure to enter AWS credentials in the cell when prompted

In [ ]:
#@title Install Python Package

!pip install git+https://github.com/RebootMotion/reboot-toolkit.git@jimmy/add_mujoco > /dev/null
!pip install git+https://github.com/RebootMotion/mlb-statsapi.git@v1.1.0#egg=mlb_statsapi > /dev/null
!echo "Done Installing"

In [ ]:
#@title Import Python Libraries

import matplotlib.pyplot as plt
import mujoco
import numpy as np
import os
import pandas as pd

from IPython.display import display
from tqdm import tqdm

import reboot_toolkit as rtk

from reboot_toolkit import S3Metadata, MocapType, MovementType, Handedness, FileType, PlayerMetadata, setup_aws, decorate_primary_segment_df_with_stats_api

In [ ]:
#@title User Input - No code changes required below this section, just enter information in forms

# Update the below info to match your desired analysis information
# Common changes you might want to make:

# To analyze both Hawk-Eye HFR data from the Stats API,
# and also Hawk-Eye Action files (e.g. from the DSP),
#  set mocap_types=[MocapType.HAWKEYE_HFR, MocapType.HAWKEYE]

# To analyze baseball-hitting,
# set movement_type=MovementType.BASEBALL_HITTING

# To analyze right-handed players,
# set handedness=Handedness.RIGHT

# To analyze data from the momentum and energy files,
# set file_type=FileType.MOMENTUM_ENERGY

# See https://docs.rebootmotion.com/ for all available file types and the data in each
mocap_types = [MocapType.HAWKEYE_HFR]
movement_type = MovementType.BASEBALL_PITCHING
handedness = Handedness.RIGHT
file_type = FileType.INVERSE_KINEMATICS

# Update the label to whatever you'd like to be displayed in the visuals
primary_segment_label = 'Primary Segment'
comparison_segment_label = 'Comparison Segment'

In [ ]:
#@title AWS Credentials

# Upload your Organization's .env file to the local file system, per https://pypi.org/project/python-dotenv/
#
# Also, update the org_id in the field below to your own org_id
# (note this isn't strictly necessary, the .env file will override what's written here)

boto3_session = setup_aws(org_id="org-mlbbiomech", aws_default_region="us-west-1")

In [ ]:
#@title Set S3 File Info

s3_metadata = S3Metadata(
    org_id=os.environ['ORG_ID'],
    mocap_types=mocap_types,
    movement_type=movement_type,
    handedness=handedness,
    file_type=file_type,
)

s3_df = rtk.download_s3_summary_df(s3_metadata)

In [ ]:
def mujoco_sim(mj_model, mj_data, mj_joint_names, positions_df, velocities_df):
    
    col_suffix = "invdyn"
    
    sim_results = []
    
    for i, row_pos in positions_df.iterrows():
        
        row_vel = velocities_df.iloc[i]
        
        mj_data.qpos = row_pos.to_numpy()
        
        mj_data.qvel = row_vel.to_numpy()
        
        mujoco.mj_step(mj_model, mj_data)
        
        mujoco.mj_inverse(mj_model, mj_data)
        
        sim_results.append(mj_data.qfrc_inverse.copy())
        
    return pd.DataFrame(
        data=sim_results, columns=[f"{jn}_{col_suffix}" for jn in mj_joint_names]
    )

In [ ]:
def simulate_segment_df(model_xml_str, segment_data_df):
    
    joint_names = rtk.get_model_info(model_xml_str, 'joint', return_names=True)
    
    model = mujoco.MjModel.from_xml_string(model_xml_str)
    
    sim_dfs = []
    
    for org_movement_id in tqdm(segment_data_df['org_movement_id'].unique()):
        
        data = mujoco.MjData(model)
        
        ik_df = segment_data_df.loc[
            segment_data_df['org_movement_id'] == org_movement_id
        ].copy().reset_index(drop=True)

        dt = ik_df['time'].diff().median()

        model.opt.timestep = dt

        ik_df = rtk.reorder_joint_angle_df_like_model(model, data, ik_df, joint_names).apply(np.radians)

        vel_df = ik_df.copy().apply(np.gradient, raw=True) / dt

        sim_dfs.append(mujoco_sim(model, data, joint_names, ik_df, vel_df))
    
    sim_df = pd.concat(sim_dfs, ignore_index=True)
    
    return pd.concat([segment_data_df, sim_df], axis=1)

In [ ]:
#@title Display the Interface for Selecting the Primary Data Segment to Analyze

# Run this cell to display the dropdown menus and reset all options to NULL
primary_segment_widget = rtk.create_interactive_widget(s3_df)
display(primary_segment_widget)

In [ ]:
#@title Set Primary Analysis Segment Info

primary_segment_data = primary_segment_widget.children[1].result
primary_analysis_segment = PlayerMetadata(
    org_player_ids=primary_segment_data["org_player_ids"],
    session_dates=primary_segment_data["session_dates"],
    session_nums=primary_segment_data["session_nums"],
    session_date_start=primary_segment_data["session_date_start"],
    session_date_end=primary_segment_data["session_date_end"],
    year=primary_segment_data["year"],
    org_movement_id=None, # set the play GUID for the skeleton animation; None defaults to the first play
    s3_metadata=s3_metadata,
)

primary_segment_summary_df = rtk.filter_s3_summary_df(primary_analysis_segment, s3_df)

primary_segment_data_df = rtk.load_games_to_df_from_s3_paths(
    primary_segment_summary_df['s3_path_delivery'].tolist(), add_ik_joints=True, add_elbow_var_val=True
)

primary_segment_data_df = decorate_primary_segment_df_with_stats_api(primary_segment_data_df)

# Common Issue:
# If no data files are returned here,
# check that the segment selection widget and the S3 File Info above are set correctly,
# also if the cells were updated after running once, check that they were run again with any new selections.

In [ ]:
primary_player_mass = 81.6

primary_model_xml_str = rtk.scale_human_xml(primary_segment_data_df, primary_player_mass, boto3_session=boto3_session)

primary_sim_data_df = simulate_segment_df(primary_model_xml_str, primary_segment_data_df)

primary_segment_dict = rtk.load_data_into_analysis_dict(
    primary_analysis_segment, primary_sim_data_df, segment_label=primary_segment_label
)

In [ ]:
#@title Display the Interface for Selecting the Comparison Data Segment to Analyze

comparison_segment_widget = rtk.create_interactive_widget(s3_df)
display(comparison_segment_widget)

In [ ]:
#@title Set Comparison Analysis Segment Inputs

comparison_s3_metadata = s3_metadata
comparison_segment_data = comparison_segment_widget.children[1].result

comparison_analysis_segment = PlayerMetadata(
    org_player_ids=comparison_segment_data["org_player_ids"],
    session_dates=comparison_segment_data["session_dates"],
    session_nums=comparison_segment_data["session_nums"],
    session_date_start=comparison_segment_data["session_date_start"],
    session_date_end=comparison_segment_data["session_date_end"],
    year=comparison_segment_data["year"],
    org_movement_id=None, # set the play GUID for the skeleton animation; None defaults to the first play
    s3_metadata=comparison_s3_metadata,
)

comparison_segment_summary_df = rtk.filter_s3_summary_df(comparison_analysis_segment, s3_df)

comparison_segment_data_df = rtk.load_games_to_df_from_s3_paths(
    comparison_segment_summary_df['s3_path_delivery'].tolist(), add_ik_joints=True, add_elbow_var_val=True
)

comparison_segment_data_df = decorate_primary_segment_df_with_stats_api(comparison_segment_data_df)

In [ ]:
comparison_player_mass = 81.6

comparison_model_xml_str = rtk.scale_human_xml(comparison_segment_data_df, comparison_player_mass, boto3_session=boto3_session)

comparison_sim_data_df = simulate_segment_df(comparison_model_xml_str, comparison_segment_data_df)

In [ ]:
comparison_segment_dict = rtk.load_data_into_analysis_dict(
    comparison_analysis_segment, comparison_sim_data_df, segment_label=comparison_segment_label
)

In [ ]:
# analysis_dicts = [primary_segment_dict]
analysis_dicts = [primary_segment_dict, comparison_segment_dict]

In [ ]:
#@title Optional - Create Simple Comparison Plots

# Available time options for the x_column include: 'time_from_max_hand', 'norm_time', 'rel_frame', 'time'
x_column = 'time_from_max_hand'

y_columns = ['right_shoulder_rot_invdyn, right_elbow_var_invdyn', 'right_elbow_flex_invdyn']

# Set the y-axis label to whatever is appropriate for the y-columns above
y_axis_label = "Torque (Nm)"

# Update to the number of standard deviations you want to shade in the plot relative to the mean
stand_devs_to_shade = 1.0

mpl_figs = []

for y_column in y_columns:

    mpl_fig = plt.figure()

    for segment_dict in analysis_dicts:

        y = segment_dict['df_mean'][y_column]

        y_lo = segment_dict['df_mean'][y_column] - (stand_devs_to_shade * segment_dict['df_std'][y_column])
        y_hi = segment_dict['df_mean'][y_column] + (stand_devs_to_shade * segment_dict['df_std'][y_column])

        plt.fill_between(segment_dict['df_mean'][x_column], y_lo, y_hi, alpha=0.4)

        plt.plot(segment_dict['df_mean'][x_column], y, label=segment_dict['segment_label'])

    plt.ylabel(y_axis_label)

    plt.xlabel(x_column)
    # plt.xlim([-1, 0.1])  # uncomment to limit the width of the x-axis

    plt.title(y_column)

    plt.legend()

    plt.grid()

    plt.show()

    mpl_figs.append(mpl_fig)

In [ ]:
#@title Optional - Save Plots to a PDF

from matplotlib.backends.backend_pdf import PdfPages

pdf_file_name = 'analysis.pdf'

pdf_analysis = PdfPages(pdf_file_name)

for mpl_fig in mpl_figs:

    pdf_analysis.savefig(mpl_fig)

pdf_analysis.close()

print('Saved plots to', pdf_file_name)